# Execute metagenome taxonomic profile
Jacobo de la Cuesta-Zuluaga. August 2024.

The aim of this notebook is to perform quality control of raw metagenome reads and run taxprofiler pipeline


Note: the human (and other eukaryote) genomes were obtained from https://doi.org/10.5281/zenodo.4629921.

 

## Before we start

The execution of the pipeline requires `conda` to be installed and an environment with `nextflow` available. You can find instructions about how to install conda [here](https://conda.io/projects/conda/en/latest/user-guide/install/index.html).

In addition, we'll be using the `nf-core` pipeline `taxprofiler`. You can read more about `nf-core` [here](https://nf-co.re/), as well as exploring the [pipeline's documentation](https://nf-co.re/taxprofiler/1.1.8/).  

Be sure to add the following to your `~/.bashrc` file. With that, we're specifying a centralized folder where the `nf-core` pipelines will be downloaded. Make sure to modify the path as required.
```
export NXF_SINGULARITY_CACHEDIR="/mnt/lustre/groups/maier/YOUR_M3HPC_USERNAME/bin/nf-core"
```


## Load libraries and set paths

First, we'll set up the libraries and the work directory where we'll save our files

In [1]:
# Libraries
library(tidyverse)
library(conflicted)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Solve conflicts
conflicts_prefer(dplyr::filter)

[conflicted] Will prefer dplyr::filter over any other package.


The following chunk will define the directories where the data is stored and where the output will be saved. The present example assumes everything will be contained in the same directory: `base_dir`. This might be different in your particular case, for example, if your sequences are stored on a centralized directory or you have multiple runs stored in different folders. You can change this accordingly. 

An example of how to combine data from multiple runs will be provided below.

In [3]:
# Directories
# Base directory
base_dir = "/mnt/lustre/groups/maier/maide581/projects/Metemgee"

# Data
data_dir = file.path(base_dir, "data")
dir.create(data_dir)

# Sequences
seq_dir = file.path(data_dir, "raw_sequences")
dir.create(seq_dir)

# Out
out_dir = file.path(data_dir, "taxprofiler")
dir.create(out_dir)

# sheets dir
sheets_dir = file.path(data_dir, "sheets")
dir.create(sheets_dir)

# Software
conda_env = "nextflow"

Warning message in dir.create(data_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Metemgee/data' already exists”
Warning message in dir.create(seq_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences' already exists”
Warning message in dir.create(sheets_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/sheets' already exists”


## Download test files

For the present example, we'll use publicly available metagenome files. They correspond to multiple sequencing runs of two samples, this means that the same sample was sequenced multiple times to achieve the desired sequencing depth. We'll use these samples to illustrate how to merge multiple sequencing runs to obtain the metagenome profile of a single sample.

In [4]:
# URL of the files from the ENA
example_fastqs = c("ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880517/MI-142-H.R1.RUN0129.L7.fastq.gz", 
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880518/MI-142-H.R1.RUN0118.L4.fastq.gz",  
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880517/MI-142-H.R2.RUN0129.L7.fastq.gz", 
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880518/MI-142-H.R2.RUN0118.L4.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880579/MI-237-H.R2.RUN0129.L7.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880581/MI-237-H.R1.RUN0118.L2.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880577/MI-237-H.R1.RUN0173.L6.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880582/MI-237-H.R1.RUN0102.L5.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880579/MI-237-H.R1.RUN0129.L7.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880581/MI-237-H.R2.RUN0118.L2.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880577/MI-237-H.R2.RUN0173.L6.fastq.gz",
    "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880582/MI-237-H.R2.RUN0102.L5.fastq.gz")

example_fastqs

[1] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880517/MI-142-H.R1.RUN0129.L7.fastq.gz"
 [2] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880518/MI-142-H.R1.RUN0118.L4.fastq.gz"
 [3] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880517/MI-142-H.R2.RUN0129.L7.fastq.gz"
 [4] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880518/MI-142-H.R2.RUN0118.L4.fastq.gz"
 [5] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880579/MI-237-H.R2.RUN0129.L7.fastq.gz"
 [6] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880581/MI-237-H.R1.RUN0118.L2.fastq.gz"
 [7] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880577/MI-237-H.R1.RUN0173.L6.fastq.gz"
 [8] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880582/MI-237-H.R1.RUN0102.L5.fastq.gz"
 [9] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880579/MI-237-H.R1.RUN0129.L7.fastq.gz"
[10] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880581/MI-237-H.R2.RUN0118.L2.fastq.gz"
[11] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880577/MI-237-H.R2.RUN0173.L6.fastq.gz"
[12] "ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR108/ERR10880582/MI-237-H.R2.RUN0102.L5.fastq.gz"

In [5]:
# Download files
# This will take a few minutes
map(example_fastqs, function(url){
    download.file(url = url, destfile = file.path(seq_dir, basename(url)), method = "wget")
})

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

[[8]]
[1] 0

[[9]]
[1] 0

[[10]]
[1] 0

[[11]]
[1] 0

[[12]]
[1] 0

## Create Samples file

We need to tell the pipeline which files correspond to which samples, to which sequencing run they correspond and where in our machine are stored those files. We do this by creating table where we specify the sample name and the location of the forward and reverse `fastq` files. 

In addition, we need to tell the pipeline which sequencing technology we used. In this case, we used an _Illumina_ machine, meaning that we got short reads. The specific software used by the pipeline will change according to this.

**Note** that you can create this table by hand using excel or a text editor program, and exporting it as a `csv` file. In this example we're doing this programatically to use the information of the sample name from the full path of the files.

In [6]:
# List raw sequences
raw_seq_list = list.files(seq_dir,  
        pattern = "fastq.gz",
        full.names = TRUE)
# Forward reads
forward_reads = raw_seq_list %>%
    str_subset("R1")
# Reverse reads
reverse_reads = raw_seq_list %>%
    str_subset("R2")

If your sequencing data is stored in multiple folders, you can concatenate multiple calls to `list.files()`, for example:

```
# Define dirs
seq_dir_1 = "/PATH/TO/DIR_1"
seq_dir_2 = "/PATH/TO/DIR_2"

# List files
raw_seq_list_1 = list.files(seq_dir_1,  
        pattern = "fastq.gz",
        full.names = TRUE)

raw_seq_list_2 = list.files(seq_dir_1,  
        pattern = "fastq.gz",
        full.names = TRUE)

# Combine
raw_seq_list = c(raw_seq_list_1, raw_seq_list_2)
```

Then, you can continue with separating the forward and reverse files as in the second half of the chunk above

In [7]:
# Create a single data frame for taxprofiler
reads_tax_df = data.frame(fastq_1 = forward_reads, # Full path of forward reads
        fastq_2 = reverse_reads, # Full path of reverse reads
        instrument_platform = "ILLUMINA", # Sequencing platform 
        fasta = "") %>% # Empty field since we don't have fasta files
    mutate(sample = basename(fastq_1), # Sample name from the file
        sample = str_remove(sample, "\\.R1.*")) %>%
    group_by(sample) %>%
    mutate(run_accession = str_c("run_", row_number())) %>% # If more than one run, specify which run it was
    relocate(sample, instrument_platform, run_accession) # Reorder columns

reads_tax_df %>%
    head()

sample,instrument_platform,run_accession,fastq_1,fastq_2,fasta
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
MI-142-H,ILLUMINA,run_1,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-142-H.R1.RUN0118.L4.fastq.gz,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-142-H.R2.RUN0118.L4.fastq.gz,
MI-142-H,ILLUMINA,run_2,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-142-H.R1.RUN0129.L7.fastq.gz,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-142-H.R2.RUN0129.L7.fastq.gz,
MI-237-H,ILLUMINA,run_1,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R1.RUN0102.L5.fastq.gz,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R2.RUN0102.L5.fastq.gz,
MI-237-H,ILLUMINA,run_2,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R1.RUN0118.L2.fastq.gz,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R2.RUN0118.L2.fastq.gz,
MI-237-H,ILLUMINA,run_3,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R1.RUN0129.L7.fastq.gz,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R2.RUN0129.L7.fastq.gz,
MI-237-H,ILLUMINA,run_4,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R1.RUN0173.L6.fastq.gz,/mnt/lustre/groups/maier/maide581/projects/Metemgee/data/raw_sequences/MI-237-H.R2.RUN0173.L6.fastq.gz,


In [8]:
# Write samples file
samples_sheet_tax = file.path(sheets_dir, "samples_file_taxprofiler.csv")
write_csv(reads_tax_df,file = samples_sheet_tax)

## Run pipeline

The pipeline requires a file with the location of the databases for the software to be used. 

You don't need to modify this if you are using the centralized database folder of A.G. Maier

In [9]:
# Create dbs file
dbs_df = data.frame(tool = c("kraken2","bracken", "metaphlan", "motus"),
    db_name = c("k2_standard_16gb", "B_standard_16gb", "metaphlan", "db_mOTU"),
    db_params = c("", ";-r 150", "", ""),
    db_path = c("/mnt/lustre/groups/maier/databases/Kraken_Bracken/k2_standard_16gb/k2_standard_16gb_20240605.tar.gz",
                "/mnt/lustre/groups/maier/databases/Kraken_Bracken/k2_standard_16gb/k2_standard_16gb_20240605.tar.gz",
                "/mnt/lustre/groups/maier/databases/Metaphlan",
                "/mnt/lustre/groups/maier/databases/mOTUs/db_mOTU")) 

dbs_df

# Write file
dbs_file = file.path(sheets_dir, "database_file.csv")
dbs_df %>%
    write_csv(dbs_file)

tool,db_name,db_params,db_path
<chr>,<chr>,<chr>,<chr>
kraken2,k2_standard_16gb,,/mnt/lustre/groups/maier/databases/Kraken_Bracken/k2_standard_16gb/k2_standard_16gb_20240605.tar.gz
bracken,B_standard_16gb,;-r 150,/mnt/lustre/groups/maier/databases/Kraken_Bracken/k2_standard_16gb/k2_standard_16gb_20240605.tar.gz
metaphlan,metaphlan,,/mnt/lustre/groups/maier/databases/Metaphlan
motus,db_mOTU,,/mnt/lustre/groups/maier/databases/mOTUs/db_mOTU


In [10]:
# Taxdump folder
taxdump_dir = "/mnt/lustre/groups/maier/databases/Taxdump"

# Host genomes
host_genome = "/mnt/lustre/groups/maier/databases/Host_genomes/hg19_main_mask_ribo_animal_allplant_allfungus.fa"

Next, the `nextflow` command to execute the pipeline will be created. You can modify it if you wish. As it is, it asks the pipeline to do quality control of the raw sequences and to store the clean reads. **This is important** because the clean reads can (and will) be used in downstream steps. For now, the only profiler we're using is `Kraken`+`Bracken`. You can use any of the available tools as long as you provide the corresponding database in the file above.

In [11]:
# Create command
# Base command
# To run metaphlan add  --run_metaphlan
taxprofiler_cmd = str_glue(
  "conda activate {{conda_env}} && \\
  cd {{out_dir}} && \\
  nextflow run nf-core/taxprofiler -r 1.2.3 \\
  --input {{samples_sheet}} \\
  --databases {{databases_sheet}} \\
  --outdir {{out_dir}} \\
  -profile m3c \\
  --perform_shortread_qc \\
  --perform_shortread_hostremoval \\
  --perform_runmerging \\
  --shortread_qc_dedup \\
  --save_analysis_ready_fastqs \\
  --hostremoval_reference {{host_genome}} \\
  --run_profile_standardisation \\
  --taxpasta_taxonomy_dir {{tax_dir}} \\
  --taxpasta_add_name \\
  --taxpasta_add_rank \\
  --taxpasta_add_lineage \\
  --taxpasta_add_ranklineage \\
  --run_kraken2 \\
  --run_bracken")

Once constructed, you can run the pipeline with the following command. You can simply copy and paste it on the terminal. It is wise to execute using `screen` or `tmux` so it runs in the background, so even if you disconnect from the HPC, the execution will continue.

In [12]:
# Fill command
Clean_tax_cmd = str_glue(taxprofiler_cmd,
                         conda_env = conda_env,
                         samples_sheet = samples_sheet_tax,
                         databases_sheet = dbs_file,
                         tax_dir = taxdump_dir,
                         host_genome = host_genome,
                         out_dir = out_dir)

Clean_tax_cmd

conda activate nextflow && cd /mnt/lustre/groups/maier/maide581/projects/Metemgee/data/taxprofiler && nextflow run nf-core/taxprofiler -r 1.2.3 --input /mnt/lustre/groups/maier/maide581/projects/Metemgee/data/sheets/samples_file_taxprofiler.csv --databases /mnt/lustre/groups/maier/maide581/projects/Metemgee/data/sheets/database_file.csv --outdir /mnt/lustre/groups/maier/maide581/projects/Metemgee/data/taxprofiler -profile m3c --perform_shortread_qc --perform_shortread_hostremoval --perform_runmerging --shortread_qc_dedup --save_analysis_ready_fastqs --hostremoval_reference /mnt/lustre/groups/maier/databases/Host_genomes/hg19_main_mask_ribo_animal_allplant_allfungus.fa --run_profile_standardisation --taxpasta_taxonomy_dir /mnt/lustre/groups/maier/databases/Taxdump --taxpasta_add_name --taxpasta_add_rank --taxpasta_add_lineage --taxpasta_add_ranklineage --run_kraken2 --run_bracken

## Output
Once the pipeline is finished, you should find the tables in the `taxprofiler/taxpasta` and `taxprofiler/bracken` subdirectories of the output folder you specified. The results in both are basically the same, the difference is that `taxpasta` includes the complete taxonomic classification of each microbe found, not only the species name.

**Note** that the `kraken`+`bracken` databases used in this example include the human genome and viruses. Make sure to filter them out in your statistical analyses if you're only interested in the abundance of _Bacteria_ and _Archaea_